In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord, Galactocentric

In [2]:
# Importing all datasets
# RAVE
dr6RaveXGaia = pd.read_csv('Observational Data/dr6_x_gaiadr3.csv')
# dr6Rave = pd.read_csv('Observational Data/dr6_orbits.csv')

# Gaia
dr3GaiaOut = pd.concat([pd.read_csv('Observational Data/gaiadr3_prv.csv'), pd.read_csv('Observational Data/gaiadr3_nrv.csv')], ignore_index=True)
dr3GaiaOut['dist'] = 1/(dr3GaiaOut['parallax'] * 1e-3)
dr3GaiaIn = pd.read_csv('Observational Data/gaiadr3_sgtA_dir.csv')
dr3GaiaIn['dist'] = 1/(dr3GaiaIn['parallax'] * 1e-3)

In [3]:
def SelectColumns(df, oldCols):
    cols = ['id', 'ra', 'dec', 'plx', 'dist', 'pmra', 'pmdec', 'radvel']

    col_map = dict(zip(oldCols, cols))
    return df[oldCols].rename(columns=col_map).dropna()

# Convert from ra/dec/parallax to cartesian
def EquitorialToCartesianAndRadial(df):
    # Converts coordinates to cartesian and adds 4 extra columns of data,
    # [x, y, z, r]
    coords = SkyCoord(
        ra = df['ra'].values * u.deg,
        dec = df['dec'].values * u.deg,
        distance = np.abs(df['dist'].values) * u.pc,
        pm_ra_cosdec = df['pmra'].values * u.mas/u.yr,
        pm_dec = df['pmdec'].values * u.mas/u.yr,
        radial_velocity = df['radvel'].values * u.km/u.s,
        frame = 'icrs'
    )

    galcen = coords.transform_to(Galactocentric())

    df['x'] = galcen.x.to(u.m).value
    df['y'] = galcen.y.to(u.m).value
    df['z'] = galcen.z.to(u.m).value

    return df


In [4]:
# Make data have same columns
raveXGaisCols = ['source_id','ra','dec','parallax','dist','pmra','pmdec','dr2_radial_velocity']
gaiaCols = ['source_id','ra','dec','parallax','dist','pmra','pmdec','radial_velocity']

# Combine
raveXGaia = SelectColumns(dr6RaveXGaia, raveXGaisCols)
gaiaOut = SelectColumns(dr3GaiaOut, gaiaCols)
gaiaIn = SelectColumns(dr3GaiaIn, gaiaCols)
obsData = pd.concat([raveXGaia, gaiaIn, gaiaOut], ignore_index=True)

In [5]:
obsData = EquitorialToCartesianAndRadial(obsData)
obsData['r'] = np.linalg.norm(obsData[['x','y','z']].values, axis = 1)

In [8]:
obsData.to_csv("Observational Data/RaveGaiaDataCleaned.csv", index=False)